# **Import Library**

In [1]:
import pandas as pd
import tensorflow as tf

Dataset listrik 

In [2]:
df = pd.read_csv('https://drive.google.com/uc?id=1AZRfFoyekqSYpri5183RmJjciRGz_ood', sep=',',
                     infer_datetime_format=True, index_col='datetime', header=0)
df

C:\Users\espej\AppData\Local\Temp\ipykernel_16332\513246612.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv('https://drive.google.com/uc?id=1AZRfFoyekqSYpri5183RmJjciRGz_ood', sep=',',


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
datetime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0
...,...,...,...,...,...,...,...
2007-02-14 17:19:00,0.636,0.140,241.16,2.6,0.0,0.0,0.0
2007-02-14 17:20:00,0.552,0.000,240.46,2.2,0.0,0.0,0.0
2007-02-14 17:21:00,0.538,0.000,239.74,2.2,0.0,0.0,0.0


Preprosessing

In [3]:
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data
data = df.values
data = normalize_series(data, data.min(axis=0), data.max(axis=0))

In [4]:
N_FEATURES = len(df.columns)

memisahkan data latih dan data uji

In [ ]:
SPLIT_TIME = int(len(data) * 0.5) #50% latih dan 50% uji
x_train = data[:SPLIT_TIME]
x_valid = data[SPLIT_TIME:]

membuat window

In [7]:
BATCH_SIZE = 32
N_PAST = 24
N_FUTURE = 24
SHIFT = 1
# Kode untuk membuat windowed datasets
train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)
valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)

In [8]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, input_shape=(N_PAST, N_FEATURES)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(N_FEATURES)
    ])

c:\Users\espej\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if (logs.get('mae') < 0.055 and logs.get('val_mae') < 0.055):
                self.model.stop_training = True
 
callbacks = myCallback()

In [11]:
#Kode untuk melakukan menyusun struktur sesuai dengan machine learning
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3) 
model.compile(loss='mae',
                  optimizer= optimizer,
                  metrics=["mae"])

training

In [12]:
model.fit(train_set,
          validation_data=(valid_set),
          epochs=100,
          callbacks=callbacks,
          verbose=1
    )

Epoch 1/100
   1345/Unknown 6s 4ms/step - loss: 0.0783 - mae: 0.0783

c:\Users\espej\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\trainers\epoch_iterator.py:164: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


1349/1349 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - loss: 0.0669 - mae: 0.0669 - val_loss: 0.0632 - val_mae: 0.0632
Epoch 2/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - loss: 0.0605 - mae: 0.0605 - val_loss: 0.0622 - val_mae: 0.0622
Epoch 3/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - loss: 0.0588 - mae: 0.0588 - val_loss: 0.0580 - val_mae: 0.0580
Epoch 4/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - loss: 0.0582 - mae: 0.0582 - val_loss: 0.0596 - val_mae: 0.0596
Epoch 5/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - loss: 0.0581 - mae: 0.0581 - val_loss: 0.0564 - val_mae: 0.0564
Epoch 6/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - loss: 0.0578 - mae: 0.0578 - val_loss: 0.0579 - val_mae: 0.0579
Epoch 7/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - loss: 0.0576 - mae: 0.0576 - val_loss: 0.0583 - val_mae: 0.0583
Epoch 8/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - loss: 0.0573 - mae: 0.0573 - val_loss: 0.0575 - val_mae: 0.0575
Epoch 9/100
1349/1349 ━━━━━━━━━━━━━━

melakukan prediksi

In [14]:
train_pred = model.predict(train_set)
train_pred[0][0]

      1/Unknown 0s 22ms/step

1349/1349 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step


array([ 0.38680267,  0.18928689,  0.04228766,  0.36946654, -0.01384506,
        0.00113084,  0.8605255 ], dtype=float32)